# Snowflake SQL Alchemy

### Uploading the web-scaped 'content.csv' into snowflake using Snowflake-SQLAlchemy into our created db

https://colab.research.google.com/drive/1ypw7U38NmmvOOX6EyAw5e-cj0nC-BCmW

In [ ]:
pip install python-dotenv

In [ ]:
pip install bcrypt

In [ ]:
pip install snowflake-sqlalchemy

In [7]:
#!/usr/bin/env python
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bcrypt
import os

In [8]:
create_database_query = "CREATE OR REPLACE DATABASE cfa_refresher;"
create_table_query = """CREATE OR REPLACE TABLE cfa_courses (
        name_of_topic STRING,
        year STRING,
        level STRING,
        introduction STRING,
        learning_outcomes STRING,
        link_to_summary_page STRING,
        link_to_pdf STRING
        );"""

In [9]:
create_stage = """CREATE STAGE STAGING DIRECTORY = ( ENABLE = true );"""
upload_to_stage = """PUT file://content.csv @cfa_refresher.public.STAGING;"""

In [10]:
create_warehouse = """CREATE OR REPLACE WAREHOUSE cfa_data_wh WITH
   WAREHOUSE_SIZE='X-SMALL'
   AUTO_SUSPEND = 180
   AUTO_RESUME = TRUE
   INITIALLY_SUSPENDED=TRUE;
   """

In [11]:
copy_stage_to_table = """COPY INTO cfa_courses
  FROM @cfa_refresher.public.STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"')
  PATTERN = 'content.csv.gz'
  ON_ERROR = 'skip_file';"""

In [15]:
load_dotenv()

u=os.getenv("SNOWFLAKE_USER")
p=os.getenv("SNOWFLAKE_PASS")
ai=os.getenv("SNOWFLAKE_ACCOUNTID")


engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=u,
        password=p,
        account_identifier=ai,
    )
)



try:
    connection = engine.connect()
    results = connection.execute(create_database_query)
    results = connection.execute(create_table_query)
    results = connection.execute(create_warehouse)
    connection.execute("USE DATABASE cfa_refresher")
    results = connection.execute(create_stage)
    results = connection.execute(upload_to_stage)
    connection.execute("USE WAREHOUSE cfa_data_wh")
    results = connection.execute(copy_stage_to_table)

finally:
    print("Done")
    connection.close()
    engine.dispose()

Done
